### Spelling Corrector
The four parts of the program are:

**1. Selection Mechanism:** In Python, max with a key argument does 'argmax'. 

**2. Candidate Model:** First a new concept: a simple edit to a word is a deletion (remove one letter), a transposition (swap two adjacent letters), a replacement (change one letter to another) or an insertion (add a letter). The function edits1 returns a set of all the edited strings (whether words or not) that can be made with one simple edit:

In [20]:
def edits1(word):
    "All edits that are one edit away from 'word'."
    letters = 'abcdefghijklmnopqrstuvwxyz'
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    deletes = [L + R[1:] for L, R in splits if R]
    transposes =  [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces = [L + c + R[1:] for L, R in splits if R for c in letters]
    inserts = [L + c + R for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

This can be a big set. For a word of length n, there will be n deletions, n-1 transpositions, 26n alterations, and 26(n+1) insertions, for a total of 54n+25 (of which a few are typically duplicates). For example,

In [22]:
print(len(edits1('somthing')))

442


However, if we restrict ourselves to words that are known—that is, in the dictionary— then the set is much smaller:

In [26]:
from collections import Counter
import re

def words(text): 
    return re.findall(r'\w+', text.lower())

WORDS = Counter(words(open('big.txt').read()))

def known(words):
    return set(w for w in words if w in WORDS)
print(known(edits1('somthing')))

{'something', 'soothing'}


We'll also consider corrections that require two simple edits. This generates a much bigger set of possibilities, but usually only a few of them are known words:

In [30]:
def edits2(word):
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

print(len(set(edits2('something'))))
print(known(edits2('something')))
print(known(edits2('somthing')))

114324
{'seething', 'smoothing', 'soothing', 'something'}
{'loathing', 'smoothing', 'scathing', 'nothing', 'sorting', 'seething', 'soothing', 'something'}


We say that the results of edits2(w) have an edit distance of 2 from w.

**3.Language Model:** We can estimate the probability of a word, P(word), by counting the number of times each word appears in a text file of about a million words, big.txt. It is a concatenation of public domain book excerpts from Project Gutenberg and lists of most frequent words from Wiktionary and the British National Corpus. The function words breaks text into words, then the variable WORDS holds a Counter of how often each word appears, and P estimates the probability of each word, based on this Counter. 

In [44]:
def P(word, N=sum(WORDS.values())): 
    return WORDS[word] / N

print(len(WORDS))

print(sum(WORDS.values()))

print(WORDS.most_common(10))

print(max(WORDS, key=P))

print(P('the'))

print(P('outrivaled'))

print(P('unmentioned'))

32198
1115585
[('the', 79809), ('of', 40024), ('and', 38312), ('to', 28765), ('in', 22023), ('a', 21124), ('that', 12512), ('he', 12401), ('was', 11410), ('it', 10681)]
the
0.07154004401278254
8.963906829152417e-07
0.0


We can see that there are 32,192 distinct words, which together appear 1,115,504 times, with 'the' being the most common word, appearing 79,808 times (or a probability of about 7%) and other words being less probable:

** 4. Error Model ** When author started to write this program, sitting on a plane in 2007, he had no data on spelling errors, and no internet connection (He know that may be hard to imagine today). Without data he couldn't build a good spelling error model, so he took a shortcut: He defined a trivial, flawed error model that says all known words of edit distance 1 are infinitely more probable than known words of edit distance 2, and infinitely less probable than a known word of edit distance 0. So we can make candidates(word) produce the first non-empty list of candidates in order of priority:  
1. The original word, if it is known; otherwise
2. The list of known words at edit distance one away, if there are any; otherwise
3. The list of known words at edit distance two away, if there are any; otherwise
4. The original word, even though it is not known.  

Then we don't need to multiply by a P(w|c) factor, because every candidate at the chosen priority will have the same probability (according to our flawed model). That gives us:

In [54]:
def correction(word):
    return max(candidates(word), key=P)
def candidates(word):
    return known([word]) or known(edits1(word)) or known(edits2(word)) or [word]

## Evaluation

In [58]:
def unit_tests():
    assert correction('speling') == 'spelling'              # insert
    assert correction('korrectud') == 'corrected'           # replace 2
    assert correction('bycycle') == 'bicycle'               # replace
    assert correction('inconvient') == 'inconvenient'       # insert 2
    assert correction('arrainged') == 'arranged'            # delete
    assert correction('peotry') =='poetry'                  # transpose
    assert correction('peotryy') =='poetry'                 # transpose + delete
    assert correction('word') == 'word'                     # known
    assert correction('quintessential') == 'quintessential' # unknown
    assert words('This is a TEST.') == ['this', 'is', 'a', 'test']
    assert Counter(words('This is a test. 123; A TEST this is.')) == (Counter({'123': 1, 'a': 2, 'is': 2, 'test': 2, 'this': 2}))
    assert len(WORDS) == 32192
    assert sum(WORDS.values()) == 1115504
    assert WORDS.most_common(10) == [('the', 79808), ('of', 40024), ('and', 38311), ('to', 28765), ('in', 22020), ('a', 21124), ('that', 12512), ('he', 12401), ('was', 11410), ('it', 10681)]
    assert WORDS['the'] == 79808
    assert P('quintessential') == 0
    assert 0.07 < P('the') < 0.08
    return 'unit_tests pass'

def spelltest(tests, verbose=False):
    "Run correction(wrong) on all (right, wrong) pairs; report results."
    import time
    start = time.clock()
    good, unknown = 0, 0
    n = len(tests)
    for right, wrong in tests:
        w = correction(wrong)
        good += (w == right)
        if w != right:
            unknown += (right not in WORDS)
            if verbose:
                print('correction({}) => {} ({}); expected {} ({})'
                      .format(wrong, w, WORDS[w], right, WORDS[right]))
    dt = time.clock() - start
    print('{:.0%} of {} correct ({:.0%} unknown) at {:.0f} words per second '
          .format(good / n, n, unknown / n, n / dt))
    
def Testset(lines):
    "Parse 'right: wrong1 wrong2' lines into [('right', 'wrong1'), ('right', 'wrong2')] pairs."
    return [(right, wrong)
            for (right, wrongs) in (line.split(':') for line in lines)
            for wrong in wrongs.split()]

print(unit_tests())
spelltest(Testset(open('spell-testset1.txt'))) # Development set
spelltest(Testset(open('spell-testset2.txt'))) # Final test set

AssertionError: 